In [45]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import r2_score, mean_absolute_error

In [46]:
import warnings
warnings.filterwarnings("ignore")

# Load Dataset

In [47]:
df= pd.read_csv(r"C:\Users\harsh\Downloads\cars_us_2022 - cars_us_2022.csv")

In [48]:
pd.set_option("display.max_columns",100)

# Data Inspection/ understanding

In [49]:
df.head()

,Unnamed: 0,Brand,Model.Number,Price,Photo,Engine.Type,Body.Type,Fuel.Type,Gearbox.Type,Power.hp,Displacement.l,Torque.lbft,Cylinders,Drivetrain,MPG.City,MPG.Highway,Seats,Doors,Height.in,Length.in,Width.in,Wheelbase.in,Clearance.in,Air.Conditioner,Heater,AntiLock.Braking.System,Central.Locking,Power.Steering,Power.Windows,Leather.Seats,Anti.Lock.Braking,Brake.Assist,Central.Locking.1,Power.Door.Locks,Child.Safety.Locks,Night.Rear.View.Mirror,Rear.Seat.Belts,Door.Ajar.Warning,Adjustable.Seats,Crash.Sensor,Engine.Check.Warning,Rear.Camera,Anti.Theft.Device,Touch.Screen,Audio.System.Remote.Control,Speakers.Front...back,Bluetooth.Connectivity,USB...Auxiliary.Input,Adjustable.Steering.Column,Height.Adjustable.Driving.Seat,Digital.Clock,Smoke.Headlamps,Electric.Folding.Rear.View.Mirror,Automatic.Climate.Control,Low.Fuel.Warning.Light,Parking.Sensors,Bottle.Holder,Voice.Control,Seat.Belt.Warning,Vehicle.Stability.Control.System,Engine.Immobilizer,Rear.Reading.Lamp,Radio,Outside.Temperature.Display,Fog.Lights.Front...Back,Centrally.Mounted.Fuel.Tank,CD.Player,DVD.Player,Electric.Adjustable.Seats,Keyless.Entry,Tyre.Pressure.Monitor,Smart.Access.Card.Entry,Xenon.Headlamps,Halogen.Headlamps,Removable.Convertible.Top,Cassette.Player
0,1,Honda,Accord Hybrid Sport-L 2023,31670.0,https://www.ccarprice.com/products/Honda_Accor...,2.0L Inline-4 Hybrid,Sedan,Hybrid,CVT,204.0,2.0,247.0,4.0,front,44.0,41.0,5.0,4.0,57.1,196.1,73.3,111.4,5.8,Yes,Yes,Yes,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,NaN,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Honda,Accord Sedan 2023,28000.0,https://www.ccarprice.com/products/Honda_Accor...,1.5L Turbo Inline-4 Gas,Sedan,Petrol,CVT,192.0,1.5,260.0,4.0,front,30.0,38.0,5.0,4.0,57.1,196.1,73.3,111.4,NaN,Yes,Yes,Yes,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,NaN,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Honda,ZR-V 2023,25000.0,https://www.ccarprice.com/products/Honda_ZRV_2...,2.0L Inline-4,SUV,Petrol,CVT,158.0,2.0,138.0,4.0,front,26.0,32.0,5.0,4.0,63.4,179.8,72.4,104.5,NaN,Yes,Yes,Yes,Yes,Yes,Yes,NaN,Yes,NaN,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,NaN,Yes,Yes,NaN,Yes,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Honda,HR-V VTi-S 2022,27800.0,https://www.ccarprice.com/products/Honda_HR-V_...,2.0L Inline-4 Gas,SUV,Petrol,CVT,119.0,2.0,107.0,4.0,front,13.0,15.0,5.0,4.0,64.0,174.0,72.0,104.0,8.0,Yes,Yes,Yes,Yes,Yes,Yes,NaN,Yes,NaN,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,NaN,Yes,Yes,NaN,Yes,NaN,NaN,Yes,Yes,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Honda,HR-V VTi 2022,26900.0,https://www.ccarprice.com/products/Honda_HR-V_...,2.0L Inline-4 Gas,SUV,Petrol,CVT,119.0,2.0,107.0,4.0,front,13.0,15.0,5.0,4.0,64.0,174.0,72.0,104.0,8.0,Yes,Yes,Yes,Yes,Yes,Yes,NaN,Yes,NaN,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,NaN,Yes,Yes,NaN,Yes,NaN,NaN,Yes,Yes,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2602 entries, 0 to 2601
Data columns (total 76 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         2602 non-null   int64  
 1   Brand                              2602 non-null   object 
 2   Model.Number                       2602 non-null   object 
 3   Price                              2602 non-null   float64
 4   Photo                              2602 non-null   object 
 5   Engine.Type                        2527 non-null   object 
 6   Body.Type                          2564 non-null   object 
 7   Fuel.Type                          2602 non-null   object 
 8   Gearbox.Type                       2602 non-null   object 
 9   Power.hp                           2256 non-null   float64
 10  Displacement.l                     2023 non-null   float64
 11  Torque.lbft                        2168 non-null   float

- Data has 76 columns and 2602 rows

In [51]:
df.isnull().sum()

Unnamed: 0                      0
Brand                           0
Model.Number                    0
Price                           0
Photo                           0
                             ... 
Smart.Access.Card.Entry      2226
Xenon.Headlamps              2494
Halogen.Headlamps            2411
Removable.Convertible.Top    2365
Cassette.Player              2528
Length: 76, dtype: int64

### Check Duplicate rows

In [52]:
df.duplicated().sum()

np.int64(0)

- No duplicate data found

### Remove Unwanted columns 

In [53]:
df.drop(columns = ["Unnamed: 0","Photo","Central.Locking.1","Anti.Lock.Braking","Cassette.Player"],inplace = True)
df.drop(columns = ["Model.Number"],inplace = True)
df.drop(columns = ["Engine.Type"],inplace = True)


In [54]:
df.shape

(2602, 69)

- removed "Unnamed: 0" and "Photo" columns as they were not related to problem statement
- "Central.Locking.1" and "Anti.Lock.Braking" duplicate column
- "Cassette.Player" has only one category i.e "no"

### separate numeric and text columns and inspect them

In [55]:
Cat_cols = df.select_dtypes(include = object).columns.tolist()
Num_cols = df.select_dtypes(include = [int,float]).columns.tolist()

In [56]:
for col in Cat_cols:
    df[col] = df[col].str.strip().str.lower()

In [57]:
for col in Cat_cols:
    print(col)
    print(df[col].unique())

Brand
['honda' 'bmw' 'lexus' 'hyundai' 'toyota' 'kia' 'nissan' 'audi'
 'chevrolet' 'ford' 'mercedes' 'porsche' 'infiniti' 'jaguar' 'cadillac'
 'land rover' 'jeep' 'volkswagen' 'maserati' 'subaru' 'dodge' 'mazda'
 'chrysler' 'aston martin' 'ferrari' 'lamborghini' 'bugatti' 'bentley'
 'rolls royce' 'mclaren' 'lincoln' 'alfa romeo' 'volvo' 'mini' 'fiat'
 'acura' 'genesis' 'buick' 'gmc' 'tesla' 'rimac' 'koenigsegg' 'lotus'
 'renault' 'suzuki' 'mg' 'skoda' 'jac' 'proton' 'changan']
Body.Type
['sedan' 'suv' 'hatchback' 'van' nan 'coupe' 'convertible' 'wagon' 'truck']
Fuel.Type
['hybrid' 'petrol' 'electric' 'diesel' 'unknown']
Gearbox.Type
['cvt' 'automatic' 'manual']
Drivetrain
['front' 'all' 'rear' nan]
Air.Conditioner
['yes' nan]
Heater
['yes' nan 'no']
AntiLock.Braking.System
['yes' nan 'with ebd']
Central.Locking
['yes' nan]
Power.Steering
['yes' nan 'electric']
Power.Windows
['yes' nan 'no' 'front + rear (o)' 'front+rear']
Leather.Seats
[nan 'yes' 'no']
Brake.Assist
['yes' nan 'no']
Pow

In [58]:
# for col in Cat_cols:
#     print(df[col].value_counts(dropna = False))
#     print("-"*50)

### Clean Text Columns

In [59]:
for col in Cat_cols:
    df[col].fillna("no",inplace = True)

In [60]:
# df["Engine.Type"]= df["Engine.Type"].str.replace("no","unknown")
df["Body.Type"]= df["Body.Type"].str.replace("no","unknown")
df["Drivetrain"]= df["Drivetrain"].str.replace("no","unknown")

In [61]:
df["AntiLock.Braking.System"] = df["AntiLock.Braking.System"].str.replace("with ebd","yes")
df["Power.Steering"] = df["Power.Steering"].str.replace("electric","yes")
df["Power.Windows"] = df["Power.Windows"].str.replace("front+rear","yes")
df["Power.Windows"] = df["Power.Windows"].str.replace("front + rear (o)","yes")
df["Touch.Screen"] = df["Touch.Screen"].str.replace("8-inch touch-screen for audio, telephone, climate control, navigation, personalization & valet","yes")
df["Touch.Screen"] = df["Touch.Screen"].str.replace("10.25-inch","yes")
df["Touch.Screen"] = df["Touch.Screen"].str.replace("digital","yes")
df["Bluetooth.Connectivity"] = df["Bluetooth.Connectivity"].str.replace("bluetooth® wireless technology with touch-screen controls","yes")
df["Height.Adjustable.Driving.Seat"] = df["Height.Adjustable.Driving.Seat"].str.replace("6 way adjustable, memory function with 3 settings for driver seat","yes")
df["Low.Fuel.Warning.Light"] = df["Low.Fuel.Warning.Light"].str.replace("yers","yes")
df["Parking.Sensors"] = df["Parking.Sensors"].str.replace("rear","yes")
df["Parking.Sensors"] = df["Parking.Sensors"].str.replace("digital","yes")
df["Seat.Belt.Warning"] = df["Seat.Belt.Warning"].str.replace("driver + co driver","yes")
df["Outside.Temperature.Display"] = df["Outside.Temperature.Display"].str.replace("yyes","yes")
df["Fog.Lights.Front...Back"] = df["Fog.Lights.Front...Back"].str.replace("front","yes")
df["Xenon.Headlamps"] = df["Xenon.Headlamps"].str.replace("with led signature","yes")


In [62]:
def clean_USB(cat):
    if cat in ["+ fast usb charger","yes+ fast usb charger"]:
        return "yes"
    else:
        return cat

In [63]:
def clean_Voice(cat):
    if cat == "ye":
        return "yes"
    else:
        return cat

In [64]:
def clean_Electric(cat):
    if cat == "es":
        return "yes"
    else:
        return cat

In [65]:
df["USB...Auxiliary.Input"] = df["USB...Auxiliary.Input"].apply(clean_USB)
df["Voice.Control"] = df["Voice.Control"].apply(clean_Voice)
df["Electric.Adjustable.Seats"] = df["Electric.Adjustable.Seats"].apply(clean_Electric)

# clean Numeric columns 

In [66]:
df = df[df["Power.hp"] > 100]    

In [67]:
df = df[df["MPG.City"] > 15]  

In [68]:
df = df[df["Torque.lbft"] > 90]  

In [69]:
df = df[df["Length.in"] > 1]

In [70]:
df = df[df["Cylinders"]  < 17]

In [71]:
Cat_cols = df.select_dtypes(include = object).columns.tolist()
Num_cols = df.select_dtypes(include = [int,float]).columns.tolist()

In [72]:
# for col in Cat_cols:
#     print("Column Name : ",col)
#     print(df[col].unique())
#     print("-"*50)

In [73]:
# for col in Num_cols:
#     print("Column Name : ",col)
#     print("Minimum",df[col].min()," and Maximum",df[col].max())
#     print("-"*50)

In [74]:
for col in Num_cols:
    if df[col].isnull().sum() > 0:
        print(col, "has ",df[col].isnull().sum()," null values")

Displacement.l has  62  null values
MPG.Highway has  5  null values
Seats has  1  null values
Doors has  3  null values
Height.in has  3  null values
Wheelbase.in has  4  null values
Clearance.in has  394  null values


In [75]:
Cat_cols = df.select_dtypes(include = object).columns.tolist()
Num_cols = df.select_dtypes(include = [int,float]).columns.tolist()

# Feature Engineering

In [76]:
TARGET = "Price"

X = df.drop(columns=[TARGET])
y = df[TARGET]

In [77]:
categorical_cols = [
    "Brand","Body.Type","Fuel.Type","Gearbox.Type","Drivetrain",
    "Air.Conditioner","Heater","AntiLock.Braking.System","Central.Locking",
    "Power.Steering","Power.Windows","Leather.Seats","Brake.Assist",
    "Power.Door.Locks","Child.Safety.Locks","Night.Rear.View.Mirror",
    "Rear.Seat.Belts","Door.Ajar.Warning","Adjustable.Seats","Crash.Sensor",
    "Engine.Check.Warning","Rear.Camera","Anti.Theft.Device","Touch.Screen",
    "Audio.System.Remote.Control","Speakers.Front...back",
    "Bluetooth.Connectivity","USB...Auxiliary.Input",
    "Adjustable.Steering.Column","Height.Adjustable.Driving.Seat",
    "Digital.Clock","Smoke.Headlamps","Electric.Folding.Rear.View.Mirror",
    "Automatic.Climate.Control","Low.Fuel.Warning.Light","Parking.Sensors",
    "Bottle.Holder","Voice.Control","Seat.Belt.Warning",
    "Vehicle.Stability.Control.System","Engine.Immobilizer","Rear.Reading.Lamp",
    "Radio","Outside.Temperature.Display","Fog.Lights.Front...Back",
    "Centrally.Mounted.Fuel.Tank","CD.Player","DVD.Player",
    "Electric.Adjustable.Seats","Keyless.Entry","Tyre.Pressure.Monitor",
    "Smart.Access.Card.Entry","Xenon.Headlamps","Halogen.Headlamps",
    "Removable.Convertible.Top"
]


median_cols = ["Seats", "Doors", "Cylinders"]

mean_cols = [
    "Power.hp","Displacement.l","Torque.lbft","MPG.City","MPG.Highway",
    "Height.in","Length.in","Width.in","Wheelbase.in","Clearance.in"
]

In [78]:
class CorrelationSelector(BaseEstimator, TransformerMixin):

    def __init__(self, threshold=0.18):
        self.threshold = threshold


    def fit(self, X, y):

        X_df = pd.DataFrame(X)

        corr = X_df.apply(lambda col: np.corrcoef(col, y)[0,1])

        self.selected_features_ = corr[
            corr.abs() >= self.threshold
        ].index.tolist()

        print("Selected Features:", len(self.selected_features_))

        return self


    def transform(self, X):

        X_df = pd.DataFrame(X)

        return X_df[self.selected_features_].values

# PIPELINES

In [79]:
# Numerical pipelines
median_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

mean_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])


# Categorical pipeline
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(
        drop="first",
        handle_unknown="ignore",
        sparse_output=False
    ))
])

preprocessor = ColumnTransformer([

    ("median_num", median_pipeline, median_cols),

    ("mean_num", mean_pipeline, mean_cols),

    ("cat", cat_pipeline, categorical_cols)

])

# FULL PIPELINE


In [80]:
pipeline = Pipeline([

    ("preprocessing", preprocessor),

    ("feature_selection", CorrelationSelector(threshold=0.18)),

    ("model", LinearRegression())

])

In [81]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [82]:
pipeline.fit(X_train, y_train)

print("Training Completed")

Selected Features: 19
Training Completed


In [83]:
y_pred = pipeline.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("R2 Score:", r2)
print("MAE:", mae)

R2 Score: 0.8454657039228584
MAE: 13727.700734340244


# SAVE MODEL

In [84]:
joblib.dump(pipeline, "car_price_pipeline.pkl")

print("Pipeline saved as car_price_pipeline.pkl")

Pipeline saved as car_price_pipeline.pkl


In [85]:
joblib.dump(X.columns.tolist(), "input_columns.pkl")

['input_columns.pkl']

In [86]:
# !streamlit run app.py

In [87]:
# for col in Num_cols:
#     print(col)
#     print("min :",df[col].min(),"max :",df[col].max())

In [88]:
# @st.cache_resource
def load_model():
    return joblib.load("car_price_pipeline.pkl")

# @st.cache_resource
def load_all_columns():
    return joblib.load("input_columns.pkl")
